In [1]:
import sys
import os
import copy
import traceback
import numpy as np

########################
# Find qslam modules
########################
sys.path.append('../qslam')

from singlerun import SingleRunAnalysis
from riskanalysis import EngineeredTruth
from qslamdesignparams import GLOBALDICT


########################
# Taking in bash parameters
########################
idx_prefix = 1 # truth flag type (three options)
idx_var = 0 # lambda scan (100 options). Array index starts at 1 so subtract 1


########################
# Truth Parameters
########################

# Choose defaults to match floor case (heights didn't work)
TRUTHKWARGS = {}

BARRIER_FLOOR = 0.25*np.pi
BARRIER_HEIGHT = 0.75*np.pi
FLOOR_RATIO = 0.4 # matches floor case

TRUTHKWARGS["OneStepdheight"] = {"low": BARRIER_FLOOR, 
                                 "high": BARRIER_HEIGHT} 
TRUTHKWARGS["OneStepdfloorarea"] = FLOOR_RATIO 



########################
# Save to path 
########################

savetopath = '/scratch/QCL_KF/qslamdatapaper/' # on Artemis
# savetopath = './data/' # local

########################
# Set true field
########################

prefix_list = ['2019_Feb_1D', '2019_Feb_2D', '2019_Feb_2D_Gssn']
prefix = prefix_list[idx_prefix]

if idx_prefix == 0:
    change_gridconfig = True # 1D
    TRUTHFLAG = None # use TRUTHKWARGS
    TRUTHKWARGS["truthtype"] = 'OneStepd' 

if idx_prefix == 1:
    change_gridconfig = False # 2D
    TRUTHFLAG = None # use TRUTHKWARGS
    TRUTHKWARGS["truthtype"] = 'OneStepq' 

if idx_prefix == 2:
    change_gridconfig = False # 2D
    TRUTHFLAG = None # use TRUTHKWARGS
    TRUTHKWARGS["truthtype"] = 'Gaussian' 

########################
# Set 1D Hardware if req
########################

if change_gridconfig is True:
    
    GLOBALDICT["GRIDDICT"] = {}
    
    for idx_posy in range(25):
        
        if idx_posy < 10 :
            
            GLOBALDICT["GRIDDICT"]["QUBIT_0" + str(idx_posy)] = (0.0, float(idx_posy))
            
        if idx_posy >= 10 :
            
            GLOBALDICT["GRIDDICT"]["QUBIT_" + str(idx_posy)] = (0.0, float(idx_posy))


########################
# Set Defaults
########################

change_MAX_NUM_ITERATIONS = 75
change_MSMTS_PER_NODE = 1
change_SIGMOID_APPROX_ERROR = 10.0**(-4)
change_QUANTISATION_UNCERTY = 10.0**(-6)
change_P_ALPHA = 15
change_P_BETA = 10
change_LAMBDA_1 = 0.93
change_LAMBDA_2 = 0.77

GLOBALDICT["MODELDESIGN"]["MAX_NUM_ITERATIONS"] = change_MAX_NUM_ITERATIONS
GLOBALDICT["MODELDESIGN"]["MSMTS_PER_NODE"] = change_MSMTS_PER_NODE
GLOBALDICT["NOISEPARAMS"]["SIGMOID_APPROX_ERROR"]["SIGMA"] = change_SIGMOID_APPROX_ERROR
GLOBALDICT["NOISEPARAMS"]["QUANTISATION_UNCERTY"]["SIGMA"] = change_QUANTISATION_UNCERTY
GLOBALDICT["MODELDESIGN"]["P_ALPHA"] = change_P_ALPHA
GLOBALDICT["MODELDESIGN"]["P_BETA"] = change_P_BETA
GLOBALDICT["MODELDESIGN"]["LAMBDA_1"] = change_LAMBDA_1
GLOBALDICT["MODELDESIGN"]["LAMBDA_2"] = change_LAMBDA_2
GLOBALDICT["MODELDESIGN"]["MULTIPLER_R_MAX"] = 4.


In [2]:
numofnodes=25
truemap_generator = EngineeredTruth(numofnodes, TRUTHKWARGS)
true_map_ = truemap_generator.get_map()
repts = 50
likelihoodparams_scan = [10**-6, 10**-4, 10**-2, 0.1, 1, 10, 10**2, 10**3]
particleconfigs = [ [3,2], [9,6], [15,10], [21,14], [30, 20]]


In [ ]:
path = './data/'
prefix = '_idx_prefix_'+str(idx_prefix)+'_'

counter=0
for idx_1 in range(len(likelihoodparams_scan) -1 ): # exclude  10**3 variance
    
    for idx_2 in range(len(likelihoodparams_scan) -1): # exclude  10**3 variance
        
        GLOBALDICT["NOISEPARAMS"]["QUANTISATION_UNCERTY"]["SIGMA"] = likelihoodparams_scan[idx_1]
        GLOBALDICT["NOISEPARAMS"]["SIGMOID_APPROX_ERROR"]["SIGMA"] = likelihoodparams_scan[idx_2]
        
        fname_likelihood = str(idx_1)+'_'+str(idx_2)+'_'
        
        for idx_3 in range(len(particleconfigs)):
            
            GLOBALDICT["MODELDESIGN"]["P_ALPHA"] = particleconfigs[idx_3][0]
            GLOBALDICT["MODELDESIGN"]["P_BETA"] = particleconfigs[idx_3][1]

        
            SAMPLE_GLOBAL_MODEL = copy.deepcopy(GLOBALDICT)
            
            uniform_r_path = path+'Uni_R'+prefix+fname_likelihood+str(idx_3)
            try:
                np.load(uniform_r_path +'.npz')
            except:
                uniform_r_expt = SingleRunAnalysis(SAMPLE_GLOBAL_MODEL, true_map_, repts, beta_expansion_mode=False)
                uniform_r_expt.run_analysis(uniform_r_path)
                counter +=1
                print "I did: ", uniform_r_path
            
            trunc_r_path = path+'Trunc_R'+prefix+fname_likelihood+str(idx_3)
            try:
                np.load(trunc_r_path +'.npz')
            except:
                trunc_r_expt = SingleRunAnalysis(SAMPLE_GLOBAL_MODEL, true_map_, repts, beta_expansion_mode=True)
                trunc_r_expt.run_analysis(trunc_r_path)
                counter +=1
                print "I did: ", trunc_r_path
                
        
        




I did:  ./data/Uni_R_idx_prefix_1_0_3_0
I did:  ./data/Trunc_R_idx_prefix_1_0_3_0
I did:  ./data/Uni_R_idx_prefix_1_0_3_1
I did:  ./data/Trunc_R_idx_prefix_1_0_3_1
I did:  ./data/Uni_R_idx_prefix_1_0_3_2
I did:  ./data/Trunc_R_idx_prefix_1_0_3_2
I did:  ./data/Uni_R_idx_prefix_1_0_3_3
I did:  ./data/Trunc_R_idx_prefix_1_0_3_3
I did:  ./data/Uni_R_idx_prefix_1_0_3_4
I did:  ./data/Trunc_R_idx_prefix_1_0_3_4
I did:  ./data/Uni_R_idx_prefix_1_0_4_0
I did:  ./data/Trunc_R_idx_prefix_1_0_4_0
I did:  ./data/Uni_R_idx_prefix_1_0_4_1
I did:  ./data/Trunc_R_idx_prefix_1_0_4_1
I did:  ./data/Uni_R_idx_prefix_1_0_4_2
I did:  ./data/Trunc_R_idx_prefix_1_0_4_2
I did:  ./data/Uni_R_idx_prefix_1_0_4_3
I did:  ./data/Trunc_R_idx_prefix_1_0_4_3
I did:  ./data/Uni_R_idx_prefix_1_0_4_4
I did:  ./data/Trunc_R_idx_prefix_1_0_4_4
I did:  ./data/Uni_R_idx_prefix_1_0_5_0
I did:  ./data/Trunc_R_idx_prefix_1_0_5_0
I did:  ./data/Uni_R_idx_prefix_1_0_5_1
I did:  ./data/Trunc_R_idx_prefix_1_0_5_1
I did:  ./data/U